# Analysis of Food Store Accessibility in Erie County, New York
## Erin Gregoire
## Fall 2024

In [2]:
import pandas as pd
import sqlite3
import ast

### SQL Database Design

In this step, I have created a database called "Erie County DB" to hold all of the data for this project. This step also details the process of normalization

In [74]:
conn = sqlite3.connect('eriecountydb')
cur = conn.cursor()

In [75]:
store_data = pd.read_csv('erie_stores.csv')
income_data = pd.read_csv('erie_incomes.csv')

One challenge that was unanticipated was that there is the same zipcode but called multiple different town names or if a town name was mispelled in the original data. Here I have removed any duplicate names that have appeared. 

In [77]:
unique_pairs = []
zips = store_data['Zipcode']
cities = store_data['City']
pairs = zip(zips, cities)

for pair in pairs:
    if pair not in unique_pairs:
        unique_pairs.append(pair)

In [78]:
unique_pairs.remove((14043, 'LANCASTER'))
unique_pairs.remove((14085, 'LAKEVIEW'))
unique_pairs.remove((14150, 'TONWANDA'))
unique_pairs.remove((14206, 'CHEEKTOWAGA'))
unique_pairs.remove((14211, 'CHEEKTOWAGA'))
unique_pairs.remove((14215, 'CHEEKTOWAGA'))
unique_pairs.remove((14217, 'BUFFALO'))
unique_pairs.remove((14218, 'BUFFALO'))
unique_pairs.remove((14224, 'W SENECA'))
unique_pairs.remove((14219, 'BUFFALO'))
unique_pairs.remove((14221, 'BUFFALO'))
unique_pairs.remove((14223, 'BUFFALO'))
unique_pairs.remove((14225, 'BUFFALO'))
unique_pairs.remove((14226, 'BUFFALO'))
unique_pairs.remove((14227, 'BUFFALO'))
unique_pairs.remove((14228, 'BUFFALO'))
unique_pairs.remove((14221, 'AMHERST'))
unique_pairs.remove((14221, 'CLARENCE'))
unique_pairs.remove((14224, 'CHEEKTOWAGA'))
unique_pairs.remove((14224, 'BUFFALO'))

In [79]:
z = []
c = []
for p in unique_pairs:
    z.append(p[0])
    c.append(p[1])

#### Location Table
- Primary Key: Zipcode
- 1NF: Removed duplicated of town names so that data is atomic
- 2NF/3NF: N/A

In [81]:
cur.execute("DROP TABLE IF EXISTS Location")

In [82]:
cur.execute('''
        CREATE TABLE Location (
        Zipcode TEXT PRIMARY KEY,
        City TEXT)
        ''')

In [83]:
cur.executemany("""INSERT INTO Location (Zipcode, City) VALUES (?, ?)""", zip(z, c))

In [84]:
pd.read_sql_query('''SELECT * FROM Location''', conn)

,Zipcode,City
0,14202,BUFFALO
1,14001,AKRON
2,14213,BUFFALO
3,14220,BUFFALO
4,14222,BUFFALO
...,...,...
58,14070,GOWANDA
59,14219,BLASDELL
60,14025,BOSTON
61,14134,SARDINA


#### Establishment Type Table
- Primary Key: Establishment Code (Est_Code)
- 3NF: Establishment Type was given its own table to store the Establishment Type Data. Without its own table, there would be transitive dependency of the Establishment Type on the Establishment Code if remained connected to the Food Store table.
- 1NF/2NF: N/A

In [86]:
with open('establishments.json', 'r', encoding='utf-8') as file:
    est_data = json.load(file)

In [87]:
cur.execute("DROP TABLE IF EXISTS Establishment_Type")

In [88]:
cur.execute('''
        CREATE TABLE Establishment_Type (
        Est_Code TEXT PRIMARY KEY,
        Est_Type TEXT
        )
        ''')

In [89]:
cur.executemany("""INSERT INTO Establishment_Type (Est_Code, Est_Type) VALUES (?, ?)""", zip(list(est_data.keys()), list(est_data.values())))

In [90]:
pd.read_sql_query('''SELECT * FROM Establishment_Type''', conn)

,Est_Code,Est_Type
0,A,Store
1,B,Bakery
2,C,Food Manufacturer
3,D,Food Warehouse
4,H,Wholesale Manufactuer
5,K,Vehicle
6,W,Farm Winery
7,Y,Slaughterhouse


#### Food Store Table
- Primary Key: Store_ID
- Foreign Key: Zipcode, Establishment Code
- 1NF: Created Store_ID due to multiple stores having the same name. Establishment codes are split into multiple columns to remove multi-valued data
- 3NF: Establishment Type removed which was dependent on Establishment Code.
- 2NF: N/A

In [92]:
cur.execute("DROP TABLE IF EXISTS Food_Store")

In [93]:
cur.execute('''
        CREATE TABLE Food_Store (
        Store_ID INTEGER PRIMARY KEY,
        Store_Name TEXT,
        Zipcode TEXT,
        Store_Focus,
        Est_Code1 TEXT,
        Est_Code2 TEXT,
        Est_Code3 TEXT,
        Est_Code4 TEXT,
        UNIQUE (Store_ID),
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode),
        FOREIGN KEY (Est_Code1) REFERENCES Establishment_Type (Est_Code),
        FOREIGN KEY (Est_Code2) REFERENCES Establishment_Type (Est_Code),
        FOREIGN KEY (Est_Code3) REFERENCES Establishment_Type (Est_Code),
        FOREIGN KEY (Est_Code4) REFERENCES Establishment_Type (Est_Code)
        )
        ''')

In [94]:
info = list(zip(store_data['Food Store Name'], store_data['Zipcode'], store_data['Establishment Type']))

In [95]:
info2 = []
for tup in info:
    lst = ast.literal_eval(tup[2])
    info2.append(lst)

In [96]:
est_code1 = []
est_code2 = []
est_code3 = []
est_code4 = []
for code in info2:
    if len(code) == 1:
        est_code1.append(code[0])
        est_code2.append(None)
        est_code3.append(None)
        est_code4.append(None)
    elif len(code) == 2:
        est_code1.append(code[0])
        est_code2.append(code[1])
        est_code3.append(None)
        est_code4.append(None)
    elif len(code) == 3:
        est_code1.append(code[0])
        est_code2.append(code[1])
        est_code3.append(code[2])
        est_code4.append(None)
    elif len(code) == 4:
        est_code1.append(code[0])
        est_code2.append(code[1])
        est_code3.append(code[2])
        est_code4.append(code[3])

In [97]:
with open('storefocus1.txt', 'r') as f:
    focus_dat = f.read()

In [98]:
focus = focus_dat.strip()
focus = focus.replace('ï»¿Convenience', 'Convenience')
focus = focus.split('\n')

In [99]:
cur.executemany("""INSERT INTO Food_Store (Store_Name, Zipcode, Store_Focus, Est_Code1, Est_Code2, Est_Code3, Est_Code4) VALUES (?, ?, ?, ?, ?, ?, ?)""", zip(store_data['Food Store Name'], store_data['Zipcode'], focus, est_code1, est_code2, est_code3, est_code4))

In [100]:
pd.read_sql_query('''SELECT * FROM Food_Store''', conn)

,Store_ID,Store_Name,Zipcode,Store_Focus,Est_Code1,Est_Code2,Est_Code3,Est_Code4
0,1,KABUL MARKET & BAKERY,14226,Convenience,A,None,None,None
1,2,MEGA BITES VENDING,14202,Other,A,C,None,None
2,3,REGANS VILLAGE DELI,14001,Convenience,A,C,None,None
3,4,GOLDEN BURMA ASIA FOODS,14213,Convenience,A,C,None,None
4,5,SOHO MARKET,14220,Other,A,C,None,None
...,...,...,...,...,...,...,...,...
1121,1122,RITE AID PHARMACY 03411,14086,Pharmacy,A,None,None,None
1122,1123,RITE AID 3521,14209,Pharmacy,A,None,None,None
1123,1124,ALS SHORT STOP GROCERY,14218,Convenience,A,C,None,None
1124,1125,CONSUMERS BEVERAGES,14075,Specialty,A,C,None,None


#### Income Tables:
- Primary Key: Zipcode
- Foreign Key: Zipcode
- 2NF: Removed partial dependencies by creating a seperate table for each income bracket
- 1NF/3NF: N/A

In [102]:
income_data.columns

Index(['Zipcode', 'Size of adjusted gross income', 'Number of returns',
       'Number of single returns', 'Number of joint returns',
       'Number of head of household returns',
       'Percentage (Number of returns)'],
      dtype='object')

In [103]:
data = list(zip(income_data['Zipcode'], income_data['Size of adjusted gross income'], income_data['Number of returns'], income_data['Percentage (Number of returns)']))

In [104]:
data2 = []
for t in data:
    if t[0] in list(store_data['Zipcode']):
        data2.append(t)

In [105]:
total = []
under25 = []
_25to50 = []
_50to75 = []
_75to100 = []
_100to200 = []
above200 = []

for i in data2:
    if i[1] == 'Total':
        total.append(i)
    elif i[1] == '$1 under $25,000':
        under25.append(i)
    elif i[1] == '$25,000 under $50,000':
        _25to50.append(i)
    elif i[1] == '$50,000 under $75,000':
        _50to75.append(i)
    elif i[1] == '$75,000 under $100,000':
        _75to100.append(i)
    elif i[1] == '$100,000 under $200,000':
        _100to200.append(i)
    elif i[1] == '$200,000 or more':
        above200.append(i)

In [106]:
zip_total = []
num_total = []
for tot in total:
    zip_total.append(tot[0])
    num_total.append(tot[2])

##### Income Total Table
- Primary Key: Zipcode
- Foreign Key: Zipcode

In [108]:
cur.execute("DROP TABLE IF EXISTS Income_Total")

In [109]:
cur.execute('''
        CREATE TABLE Income_Total (
        Zipcode TEXT PRIMARY KEY,
        No_of_Returns INTEGER,
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode)
        )
        ''')

In [110]:
cur.executemany("""INSERT INTO Income_Total (Zipcode, No_of_Returns) VALUES (?, ?)""", zip(zip_total, num_total))

In [111]:
pd.read_sql_query('''SELECT * FROM Income_Total''', conn)

,Zipcode,No_of_Returns
0,14001,4880
1,14004,5540
2,14006,4650
3,14025,1590
4,14026,450
5,14030,750
6,14031,5310
7,14032,4710
8,14033,1240
9,14034,830


##### Income Under 25k Table
- Primary Key: Zipcode
- Foreign Key: Zipcode

In [113]:
zip_under25 = []
num_under25 = []
perc_under25 = []
for u in under25:
    zip_under25.append(u[0])
    num_under25.append(u[2])
    perc_under25.append(round(u[3], 2))

In [114]:
cur.execute("DROP TABLE IF EXISTS Income_Under_25k")

In [115]:
cur.execute('''
        CREATE TABLE Income_Under_25k (
        Zipcode TEXT PRIMARY KEY,
        No_of_Returns INTEGER,
        Percentage REAL,
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode)
        )
        ''')

In [116]:
cur.executemany("""INSERT INTO Income_Under_25k (Zipcode, No_of_Returns, Percentage) VALUES (?, ?, ?)""", zip(zip_under25, num_under25, perc_under25))

In [117]:
pd.read_sql_query('''SELECT * FROM Income_Under_25k''', conn)

,Zipcode,No_of_Returns,Percentage
0,14001,1190,24.39
1,14004,1330,24.01
2,14006,1210,26.02
3,14025,340,21.38
4,14026,100,22.22
5,14030,190,25.33
6,14031,1320,24.86
7,14032,1150,24.42
8,14033,280,22.58
9,14034,210,25.30


##### Income for 25k to 50k Table
- Primary Key: Zipcode
- Foreign Key: Zipcode

In [119]:
zip_25 = []
num_25 = []
perc_25 = []
for v in _25to50:
    zip_25.append(v[0])
    num_25.append(v[2])
    perc_25.append(round(v[3], 2))

In [120]:
cur.execute("DROP TABLE IF EXISTS Income_25k_to_50k")

In [121]:
cur.execute('''
        CREATE TABLE Income_25k_to_50k (
        Zipcode TEXT PRIMARY KEY,
        No_of_Returns INTEGER,
        Percentage REAL,
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode)
        )
        ''')

In [122]:
cur.executemany("""INSERT INTO Income_25k_to_50k (Zipcode, No_of_Returns, Percentage) VALUES (?, ?, ?)""", zip(zip_25, num_25, perc_25))

In [123]:
pd.read_sql_query('''SELECT * FROM Income_25k_to_50k''', conn)

,Zipcode,No_of_Returns,Percentage
0,14001,1170,23.98
1,14004,1310,23.65
2,14006,1280,27.53
3,14025,320,20.13
4,14026,90,20.00
5,14030,190,25.33
6,14031,920,17.33
7,14032,660,14.01
8,14033,240,19.35
9,14034,200,24.10


##### Income for 50k to 75k Table
- Primary Key: Zipcode
- Foreign Key: Zipcode

In [125]:
zip_50 = []
num_50 = []
perc_50 = []
for w in _50to75:
    zip_50.append(w[0])
    num_50.append(w[2])
    perc_50.append(round(w[3], 2))

In [126]:
cur.execute("DROP TABLE IF EXISTS Income_50k_to_75k")

In [127]:
cur.execute('''
        CREATE TABLE Income_50k_to_75k (
        Zipcode TEXT PRIMARY KEY,
        No_of_Returns INTEGER,
        Percentage REAL,
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode)
        )
        ''')

In [128]:
cur.executemany("""INSERT INTO Income_50k_to_75k (Zipcode, No_of_Returns, Percentage) VALUES (?, ?, ?)""", zip(zip_50, num_50, perc_50))

In [129]:
pd.read_sql_query('''SELECT * FROM Income_50k_to_75k''', conn)

,Zipcode,No_of_Returns,Percentage
0,14001,830,17.01
1,14004,890,16.06
2,14006,820,17.63
3,14025,260,16.35
4,14026,90,20.00
5,14030,140,18.67
6,14031,680,12.81
7,14032,500,10.62
8,14033,190,15.32
9,14034,170,20.48


##### Income 75k to 100k Table
- Primary Key: Zipcode
- Foreign Key: Zipcode

In [131]:
zip_75 = []
num_75 = []
perc_75 = []
for nn in _75to100:
    zip_75.append(nn[0])
    num_75.append(nn[2])
    perc_75.append(round(nn[3], 2))

In [132]:
cur.execute("DROP TABLE IF EXISTS Income_75k_to_100k")

In [133]:
cur.execute('''
        CREATE TABLE Income_75k_to_100k (
        Zipcode TEXT PRIMARY KEY,
        No_of_Returns INTEGER,
        Percentage REAL,
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode)
        )
        ''')

In [134]:
cur.executemany("""INSERT INTO Income_75k_to_100k (Zipcode, No_of_Returns, Percentage) VALUES (?, ?, ?)""", zip(zip_75, num_75, perc_75))

In [135]:
pd.read_sql_query('''SELECT * FROM Income_75k_to_100k''', conn)

,Zipcode,No_of_Returns,Percentage
0,14001,600,12.30
1,14004,710,12.82
2,14006,550,11.83
3,14025,220,13.84
4,14026,60,13.33
5,14030,100,13.33
6,14031,520,9.79
7,14032,430,9.13
8,14033,140,11.29
9,14034,110,13.25


##### Income for 100k to 200k Table
- Primary Key: Zipcode
- Foreign Key: Zipcode

In [137]:
zip_100 = []
num_100 = []
perc_100 = []
for mm in _100to200:
    zip_100.append(mm[0])
    num_100.append(mm[2])
    perc_100.append(round(mm[3], 2))

In [138]:
cur.execute("DROP TABLE IF EXISTS Income_100k_to_200k")

In [139]:
cur.execute('''
        CREATE TABLE Income_100k_to_200k (
        Zipcode TEXT PRIMARY KEY,
        No_of_Returns INTEGER,
        Percentage REAL,
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode)
        )
        ''')

In [140]:
cur.executemany("""INSERT INTO Income_100k_to_200k (Zipcode, No_of_Returns, Percentage) VALUES (?, ?, ?)""", zip(zip_100, num_100, perc_100))

In [141]:
pd.read_sql_query('''SELECT * FROM Income_100k_to_200k''', conn)

,Zipcode,No_of_Returns,Percentage
0,14001,890,18.24
1,14004,1110,20.04
2,14006,680,14.62
3,14025,360,22.64
4,14026,90,20.00
5,14030,130,17.33
6,14031,1080,20.34
7,14032,1070,22.72
8,14033,310,25.00
9,14034,140,16.87


##### Income Above 200k Table
- Primary Key: Zipcode
- Foreign Key: Zipcode

In [143]:
zip_200 = []
num_200 = []
perc_200 = []
for zz in above200:
    zip_200.append(zz[0])
    num_200.append(zz[2])
    perc_200.append(round(zz[3], 2))

In [144]:
cur.execute("DROP TABLE IF EXISTS Income_Above_200k")

In [145]:
cur.execute('''
        CREATE TABLE Income_Above_200k (
        Zipcode TEXT PRIMARY KEY,
        No_of_Returns INTEGER,
        Percentage REAL,
        FOREIGN KEY (Zipcode) REFERENCES Location (Zipcode)
        )
        ''')

In [146]:
cur.executemany("""INSERT INTO Income_Above_200k (Zipcode, No_of_Returns, Percentage) VALUES (?, ?, ?)""", zip(zip_200, num_200, perc_200))

In [147]:
pd.read_sql_query('''SELECT * FROM Income_Above_200k''', conn)

,Zipcode,No_of_Returns,Percentage
0,14001,200,4.10
1,14004,190,3.43
2,14006,110,2.37
3,14025,90,5.66
4,14026,20,4.44
5,14030,0,0.00
6,14031,790,14.88
7,14032,900,19.11
8,14033,80,6.45
9,14034,0,0.00


In [148]:
conn.commit()
conn.close()

### Summary of SQL Database Design
- All tables have been normalized to 3NF
- Tables within the Erie County Database:
    - Location
    - Food Store
    - Establishment Type
    - Income Total
    - Income Under 25k
    - Income 25k to 50k
    - Income 50k to 75k
    - Income 75k to 100k
    - Income 100k to 200k
    - Income Above 200k